In [1]:
# import libraries
import csv
import pymongo
import pandas as pd
from pathlib import Path

In [2]:
# import csv file
salary_dataset = pd.read_excel("../Datasets/Player_salary_1990-2017.xlsx")
birthplace_dataset = pd.read_csv("../Datasets/final_heatmap.csv")

In [3]:
salary_dataset = salary_dataset.rename(columns={'Player Name':'player_name','Salary in $':'salary_$','Season Start':'season_start','Season End':'season_end','Full Team Name':'team_name'})
salary_dataset = salary_dataset.drop(axis=1,columns=['Register Value'])
salary_dataset.head()

,player_name,salary_$,season_start,season_end,Team,team_name
0,A.C. Green,1750000,1990,1991,LAL,Los Angeles Lakers
1,A.C. Green,1750000,1991,1992,LAL,Los Angeles Lakers
2,A.C. Green,1750000,1992,1993,LAL,Los Angeles Lakers
3,A.C. Green,1885000,1993,1994,PHO,Phoenix Suns
4,A.C. Green,6472600,1994,1995,PHO,Phoenix Suns


# Establish connection to MongoDB

In [3]:
# Establish connection to local host 27017
conn = 'mongodb://localhost:27017'

# Connect to MongoClient
client = pymongo.MongoClient(conn)
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [5]:
# Begin by dropping any previous version of the MongoDB instance.
client.drop_database('nba_players_db')

In [4]:
# Create database by naming db here
db = client.nba_players_db

# Create collection by naming collection here

### Salary collection

In [7]:
# Create salary collection
salary_collection = db.player_salary_info

In [8]:
# Use ".to_dict('records')"in combination with .csv variable to create list of dictionaries data set
salary_dataset.reset_index(inplace=True)

salary_dict = salary_dataset.to_dict('records')

# Insert data from the 'data_dict' variable into the collection using 'collection' variable
salary_collection.insert_many(salary_dict)

### Player WS Collection

In [9]:
# Create collection for player WS info 1976-2020
playerWS_collection = db.playerWS


In [10]:
folder = "../Data/Data_Raw"

year = 1976

counter = 0

# i = 0

for file in Path(folder).glob('*.csv'):
    df = pd.read_csv(file)
    df.drop(df[df['Tm'] == 'TOT'].index, inplace = True)
    df = df[['Player','MP','WS', 'WS/48', 'BPM']]
    df = df.groupby(['Player']).sum().reset_index()
    df['Player'] = df['Player'].str.replace('?', '')
    df['Player'] = df['Player'].str.replace('*', '')
    for index, row in df.iterrows():
        obj = {
            'year': year,
            'player': row['Player'],
            'mp': row['MP'],
            'ws': row['WS'],
            'wsper48': row['WS/48'],
            'bpm': row['BPM']
        }
    year += 1   
    counter += 1
    
    
    playerWS_collection.insert_one(obj)

#     i += 1
#     if i == 17:
#        break

In [11]:
# len(obj)
# print(counter)

### Player birthplace collection

In [12]:
player_birthplace_collection = db.playerBirthplace

In [13]:
# Use ".to_dict('records')"in combination with .csv variable to create list of dictionaries data set
birthplace_dataset.reset_index(inplace=True)

birthplace_dict = birthplace_dataset.to_dict('records')

# Insert data from the 'data_dict' variable into the collection using 'collection' variable
player_birthplace_collection.insert_many(birthplace_dict)


### Birthplace with WS and location collection

In [14]:
# Old collection, not sure if we'll be using anymore
# WS_birthplace_collection = db.WSbirthplace

In [5]:
ws_states_collection = db.ws_states

In [6]:
birthplace_dataset.head()

,Birth_Place,WS,latitude,longitude
0,Alabama,1291.3,32.318231,-86.902298
1,Alaska,28.5,63.588753,-154.493062
2,Argentina,93.2,-38.416097,-63.616672
3,Arizona,100.6,34.048928,-111.093731
4,Arkansas,714.4,35.201050,-91.831833


In [8]:
states_df = pd.read_csv('../Data/states.csv')

In [9]:
states_df

,abr,state
0,AA,Armed Forces (the) Americas
1,AB,Alberta
2,AE,Armed Forces Europe
3,AK,Alaska
4,AL,Alabama
...,...,...
71,WA,Washington
72,WI,Wisconsin
73,WV,West Virginia
74,WY,Wyoming


In [12]:
for index, row in birthplace_dataset.iterrows():
    for i, r in states_df.iterrows():
        if row['Birth_Place'] == r['state']:
            birthplace_dataset.loc[index, 'abreviation'] = r['abr']

In [13]:
birthplace_dataset

,Birth_Place,WS,latitude,longitude,abreviation
0,Alabama,1291.3,32.318231,-86.902298,AL
1,Alaska,28.5,63.588753,-154.493062,AK
2,Argentina,93.2,-38.416097,-63.616672,NaN
3,Arizona,100.6,34.048928,-111.093731,AZ
4,Arkansas,714.4,35.201050,-91.831833,AR
...,...,...,...,...,...
106,Virginia,1006.8,37.431573,-78.656894,VA
107,Washington,779.2,47.751074,-120.740139,WA
108,West Virginia,83.2,38.597626,-80.454903,WV
109,Wisconsin,438.4,43.784440,-88.787868,WI


In [14]:
# Creating an object from the DF
for index, row in birthplace_dataset.iterrows():
    post = {
        'birthplace': row['Birth_Place'],
        'win_shares': row['WS'],
        'long': row['longitude'],
        'lat': row['latitude'],
        'abr' : str(row['abreviation'])
    }
    
    ws_states_collection.insert_one(post)

In [15]:
# This is the old collection, still in the DB, but may not use anymore. 

# # Creating an object from the DF
# for index, row in birthplace_dataset.iterrows():
#     heatmap = {
#         'birthplace': row['Birth_Place'],
#         'win_shares': row['WS'],
#         'long': row['longitude'],
#         'lat': row['latitude'], 
#     }
    
#     WS_birthplace_collection.insert_one(heatmap)

In [16]:
# # CODE TO PRINT OUT CONTENTS OF THE DATABASE
# salary_collection = db['player_salary_info']

# result = salary_collection.find( {"Player Name": "LeBron James"} )
# # result

# for doc in result:
#     print (doc)
# ======================================================
# sal = salary_collection.find()
# sal

# for x in sal:
#     print(x)

In [17]:
# After finished with analysis, drop database by uncommenting and running this code
# client.drop_database('final_testing_beds_db')

In [18]:
# <<variable name>> = db.testing_beds.find({"<<field>>":"<<value>>"}).pretty()
